In [80]:
import re
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import spacy
from sklearn.decomposition import PCA

In [6]:
!python3 -m spacy download en_core_web_sm

2022-05-16 18:05:25.877256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 18:05:25.877331: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 21.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
nlp = spacy.load('en_core_web_sm')

In [2]:
!pip install vaderSentiment

In [96]:
## Importing data csv file

df = pd.read_csv('Musical_instruments_reviews.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB


In [9]:
df[['reviewText', 'overall']].head()
df['reviewText']= df['reviewText'].astype(str)

In [10]:
df['spacy_doc'] = list(nlp.pipe(df.reviewText))

df[['spacy_doc', 'overall']].head()

,spacy_doc,overall
0,"(Not, much, to, write, about, here, ,, but, it...",5.0
1,"(The, product, does, exactly, as, it, should, ...",5.0
2,"(The, primary, job, of, this, device, is, to, ...",5.0
3,"(Nice, windscreen, protects, my, MXL, mic, and...",5.0
4,"(This, pop, filter, is, great, ., It, looks, a...",5.0


In [76]:
print('rate over 3.0 is ', len(df[df.overall > 3.0]))
print('rate under or equal to  3.0 is ', len(df[df.overall <= 3.0]))
ratio_value = 9022/1239
print(ratio_value)

rate over 3.0 is  9022
rate under or equal to  3.0 is  1239
7.281678773204197


In [ ]:
hib = [(len(df[df.overall > 3.0]), )]

In [41]:
five_reviews = df[df.overall > 3.0]

five_adj = [token.text.lower() for doc in
          five_reviews.spacy_doc 
          for token in doc
          if token.pos_=='ADJ']



['much',
 'crisp',
 'lowest',
 'same',
 'affordable',
 'better',
 'added',
 'small',
 'old',
 'reminiscent',
 'next',
 'expensive',
 'pleasing',
 'primary',
 'noticeable',
 'high',
 'double',
 'secure',
 'little',
 'nice',
 'only',
 'able',
 'careful',
 'great',
 'good',
 'heavy',
 'great',
 'last',
 'great',
 'good',
 'worth',
 'simple',
 'perfect',
 'several',
 'best',
 'great',
 'wrong',
 'wide',
 'high',
 'few',
 'glad',
 'expensive',
 'expensive',
 'sure',
 'sturdy',
 'reliable',
 'pricier',
 'bottom',
 'own',
 'possible',
 'higher',
 'higher',
 'same',
 'good',
 'great',
 'midi',
 'perfect',
 'great',
 'only',
 'unscrewed',
 'sound',
 'cool',
 'nice',
 'great',
 'more',
 'indestructable',
 'electrician',
 'more',
 'third',
 'fourth',
 'light',
 'cheaper',
 'hard',
 'cheaper',
 'few',
 'extra',
 'little',
 'durable',
 'shipped-',
 'limited',
 'unshielded',
 'more',
 'better',
 'great',
 'sure',
 'cool',
 'cheap',
 'able',
 'rattle',
 'black',
 'other',
 'cool',
 'shorter',
 'red',

In [63]:
from collections import Counter

top_adj = Counter(five_adj).most_common(20)

top_data = pd.DataFrame(top_adj, columns = ['word', 'count'])

top_adj

[('great', 3728),
 ('good', 3224),
 ('other', 1841),
 ('little', 1741),
 ('more', 1520),
 ('nice', 1515),
 ('easy', 1299),
 ('better', 916),
 ('best', 835),
 ('small', 795),
 ('acoustic', 779),
 ('new', 734),
 ('few', 717),
 ('different', 700),
 ('same', 685),
 ('perfect', 683),
 ('many', 683),
 ('cheap', 636),
 ('much', 634),
 ('high', 590)]

In [39]:
one_reviews = df[df.overall <= 3.0]

one_adj = [token.text.lower() for doc in
          one_reviews.spacy_doc
          for token in doc
          if token.pos_=='ADJ']

one_top_adj = Counter(one_adj).most_common(20)

# type(one_top_adj)
one_top_adj

[('good', 482),
 ('other', 336),
 ('great', 262),
 ('more', 253),
 ('better', 227),
 ('little', 225),
 ('cheap', 193),
 ('same', 149),
 ('nice', 144),
 ('few', 125),
 ('new', 116),
 ('small', 115),
 ('much', 110),
 ('bad', 110),
 ('low', 101),
 ('first', 100),
 ('different', 99),
 ('high', 96),
 ('sure', 96),
 ('hard', 92)]

In [50]:
data = pd.DataFrame(one_top_adj, columns = ['word', 'count'])
data

,word,count
0,good,482
1,other,336
2,great,262
3,more,253
4,better,227
5,little,225
6,cheap,193
7,same,149
8,nice,144
9,few,125


In [104]:
word_list = list(top_data.word) + list(data.word)
no_dup_word_list = sorted(set(word_list), key=lambda x: word_list.index(x))
# (no_dup_word_list)


count_list = [(3728, 3224, 1841, 1741, 1520, 1515, 1299, 916, 835, 795, 779, 734, 717, 700, 685, 683, 683, 636, 634, 590, 0, 0, 0, 0, 0),
             (262, 482, 336, 225, 253, 144, 0, 227, 0, 115, 0, 116, 125, 99, 149, 0, 0, 193, 110, 96, 110, 101, 100, 96, 92)]
review_word = pd.DataFrame(count_list, columns = no_dup_word_list)
review_word.iloc[1] = review_word.iloc[1] * ratio_value

review_word = review_word.drop(['great', 'good', 'other', 'little', 'more'], axis=1)

In [105]:
pca = PCA(n_components = 2)
pca.fit(review_word)
pca_3d_components = pca.components_
pca_3d_components

array([[-0.16309917, -0.45422052,  0.25768573, -0.29197393,  0.01482355,
        -0.27239244,  0.03869957,  0.06755957,  0.00730326,  0.13985731,
        -0.23882418, -0.23882418,  0.26902303,  0.05838942,  0.03812836,
         0.28007981,  0.25716419,  0.25461801,  0.24443329,  0.23424857],
       [-0.9307301 ,  0.07580983, -0.05718253,  0.0712601 , -0.00444236,
         0.06848817, -0.17949344,  0.17967094,  0.18309038, -0.0191846 ,
         0.04531864,  0.04531864, -0.051283  , -0.00711604, -0.00841502,
        -0.0661247 , -0.05269923,  0.02873393,  0.01302932, -0.03927376]])

In [106]:
pca.transform(review_word)

array([[-1.42992220e+03,  1.71253301e-13],
       [ 1.42992220e+03,  9.87529382e-14]])

In [107]:
pca.explained_variance_ratio_

array([1.00000000e+00, 4.63484787e-32])